In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.master("local[*]").getOrCreate()

23/02/24 21:00:40 WARN Utils: Your hostname, yujaeseong-ui-MacBookAir.local resolves to a loopback address: 127.0.0.1; using 192.168.219.147 instead (on interface en0)
23/02/24 21:00:40 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/02/24 21:00:41 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
df = spark.read.option('header', 'true').csv('data/raw/green/2021/01/')


In [5]:
import pandas as pd

In [7]:
df_pd = pd.read_csv('data/raw/green/2021/01/green_tripdata_2021-01.csv.gz', nrows=1000)
df_pd.dtypes

VendorID                   int64
lpep_pickup_datetime      object
lpep_dropoff_datetime     object
store_and_fwd_flag        object
RatecodeID                 int64
PULocationID               int64
DOLocationID               int64
passenger_count            int64
trip_distance            float64
fare_amount              float64
extra                    float64
mta_tax                  float64
tip_amount               float64
tolls_amount             float64
ehail_fee                float64
improvement_surcharge    float64
total_amount             float64
payment_type               int64
trip_type                  int64
congestion_surcharge     float64
dtype: object

In [8]:
spark.createDataFrame(df_pd).schema

StructType(List(StructField(VendorID,LongType,true),StructField(lpep_pickup_datetime,StringType,true),StructField(lpep_dropoff_datetime,StringType,true),StructField(store_and_fwd_flag,StringType,true),StructField(RatecodeID,LongType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(passenger_count,LongType,true),StructField(trip_distance,DoubleType,true),StructField(fare_amount,DoubleType,true),StructField(extra,DoubleType,true),StructField(mta_tax,DoubleType,true),StructField(tip_amount,DoubleType,true),StructField(tolls_amount,DoubleType,true),StructField(ehail_fee,DoubleType,true),StructField(improvement_surcharge,DoubleType,true),StructField(total_amount,DoubleType,true),StructField(payment_type,LongType,true),StructField(trip_type,LongType,true),StructField(congestion_surcharge,DoubleType,true)))

In [9]:
from pyspark.sql import types

In [12]:
green_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("lpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("lpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("ehail_fee", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("trip_type", types.IntegerType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
    ]
)

In [13]:
df_green = spark.read.option('header', 'true').schema(green_schema).csv('data/raw/green/2021/01/')

In [14]:
df_green.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- lpep_pickup_datetime: timestamp (nullable = true)
 |-- lpep_dropoff_datetime: timestamp (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- ehail_fee: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- trip_type: integer (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [21]:
year = 2020
for month in range(1, 13):
    print(f"Processing {year}/{month:02d}...")
    
    input_path = f"data/raw/green/{year}/{month:02d}/"
    output_path = f"data/pq/green/{year}/{month:02d}/"

    df_green = spark.read \
        .option('header', 'true') \
        .schema(green_schema) \
        .csv(input_path)
        
    df_green \
        .repartition(4) \
        .write.parquet(output_path, mode='overwrite')

Processing 2020/01...


Processing 2020/02...


Processing 2020/03...


Processing 2020/04...
Processing 2020/05...


Processing 2020/06...


...

Processing 2020/08...


Processing 2020/09...


Processing 2020/10...


Processing 2020/11...


Processing 2020/12...


In [ ]:
year = 2021
for month in range(1, 9):
    print(f"Processing {year}/{month:02d}...")
    
    input_path = f"data/raw/green/{year}/{month:02d}/"
    output_path = f"data/pq/green/{year}/{month:02d}/"

    df_green = spark.read \
        .option('header', 'true') \
        .schema(green_schema) \
        .csv(input_path)
        
    df_green \
        .repartition(4) \
        .write.parquet(output_path, mode='overwrite')

Processing 2020/01...


Processing 2020/02...


Processing 2020/03...


Processing 2020/04...
Processing 2020/05...


Processing 2020/06...


Processing 2020/07...


Processing 2020/08...


In [15]:
df_yellow_pd = pd.read_csv('data/raw/yellow/2021/01/yellow_tripdata_2021-01.csv.gz', nrows=1000)

In [16]:
spark.createDataFrame(df_yellow_pd).schema

StructType(List(StructField(VendorID,LongType,true),StructField(tpep_pickup_datetime,StringType,true),StructField(tpep_dropoff_datetime,StringType,true),StructField(passenger_count,LongType,true),StructField(trip_distance,DoubleType,true),StructField(RatecodeID,LongType,true),StructField(store_and_fwd_flag,StringType,true),StructField(PULocationID,LongType,true),StructField(DOLocationID,LongType,true),StructField(payment_type,LongType,true),StructField(fare_amount,DoubleType,true),StructField(extra,DoubleType,true),StructField(mta_tax,DoubleType,true),StructField(tip_amount,DoubleType,true),StructField(tolls_amount,DoubleType,true),StructField(improvement_surcharge,DoubleType,true),StructField(total_amount,DoubleType,true),StructField(congestion_surcharge,DoubleType,true)))

In [17]:
yellow_schema = types.StructType([
    types.StructField("VendorID", types.IntegerType(), True),
    types.StructField("tpep_pickup_datetime", types.TimestampType(), True),
    types.StructField("tpep_dropoff_datetime", types.TimestampType(), True),
    types.StructField("passenger_count", types.IntegerType(), True),
    types.StructField("trip_distance", types.DoubleType(), True),
    types.StructField("RatecodeID", types.IntegerType(), True),
    types.StructField("store_and_fwd_flag", types.StringType(), True),
    types.StructField("PULocationID", types.IntegerType(), True),
    types.StructField("DOLocationID", types.IntegerType(), True),
    types.StructField("payment_type", types.IntegerType(), True),
    types.StructField("fare_amount", types.DoubleType(), True),
    types.StructField("extra", types.DoubleType(), True),
    types.StructField("mta_tax", types.DoubleType(), True),
    types.StructField("tip_amount", types.DoubleType(), True),
    types.StructField("tolls_amount", types.DoubleType(), True),
    types.StructField("improvement_surcharge", types.DoubleType(), True),
    types.StructField("total_amount", types.DoubleType(), True),
    types.StructField("congestion_surcharge", types.DoubleType(), True)
    ]
)

In [18]:
df_yellow = spark.read.option('header', 'true').schema(yellow_schema).csv('data/raw/yellow/2021/01/')

In [19]:
df_yellow.printSchema()

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp (nullable = true)
 |-- tpep_dropoff_datetime: timestamp (nullable = true)
 |-- passenger_count: integer (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: integer (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: integer (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)



In [24]:
year = 2020
for month in range(1, 13):
    print(f"Processing {year}/{month:02d}...")
    
    input_path = f"data/raw/yellow/{year}/{month:02d}/"
    output_path = f"data/pq/yellow/{year}/{month:02d}/"

    df_green = spark.read \
        .option('header', 'true') \
        .schema(yellow_schema) \
        .csv(input_path)
        
    df_green \
        .repartition(4) \
        .write.parquet(output_path, mode='overwrite')

Processing 2020/01...


Processing 2020/02...


Processing 2020/03...


Processing 2020/04...


Processing 2020/05...


...

Processing 2020/08...


Processing 2020/09...


Processing 2020/10...


Processing 2020/11...


Processing 2020/12...


In [25]:
year = 2021
for month in range(1, 9):
    print(f"Processing {year}/{month:02d}...")
    
    input_path = f"data/raw/yellow/{year}/{month:02d}/"
    output_path = f"data/pq/yellow/{year}/{month:02d}/"

    df_green = spark.read \
        .option('header', 'true') \
        .schema(yellow_schema) \
        .csv(input_path)
        
    df_green \
        .repartition(4) \
        .write.parquet(output_path, mode='overwrite')

Processing 2021/01...


Processing 2021/02...


Processing 2021/03...


Processing 2021/04...


Processing 2021/05...


Processing 2021/06...


Processing 2021/07...


Processing 2021/08...
